# Machine Learning I - Aprendizado Supervisionado
PUCRS Online - Curso de pós-graduação em Ciência de Dados e Inteligência Artificial

Prof. Martin Duarte Móre

## Material Prático - Classificação

Neste notebook, exploraremos alguns dos aspectos práticos de implementação de técnicas de Aprendizado de Máquina para a resolução de problemas de Classificação. Para tal, utilizaremos a biblioteca [`scikit-learn`](https://scikit-learn.org/stable/), que fornece a implementação de diversos algoritmos vistos durante a disciplina.

Especificamente, discutiremos os seguintes conceitos:
- Carregamento, exploração e pré-processamento de dados
- Criação de modelos de classificação
- Análise de desempenho
- Fronteira de decisão

Para executar os comandos abaixo, é necessário realizar a instalação de uma série de pacotes. Recomenda-se uma instalação completa do [Anaconda](https://www.anaconda.com/), que contém todos os pacotes necessários. Caso queira instalar os pacotes manualmente, o arquivo `environment.yaml` lista todas as dependências.

## Imports
Antes de iniciarmos, é necessário importar todos os pacotes.

In [ ]:
# carrega a extensão autoreload, que nos permite editar arquivos .py e reimportá-los automaticamente sempre que forem modificados
%load_ext autoreload
%autoreload 2

# bibliotecas instaladas
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# implementação local
import utils
import visualization as vis

## Carregando os Dados

Neste Notebook, trabalharemos com um conjunto de dados chamado [Breast Cancer Wisconsin (Diagnostic)](https://pages.cs.wisc.edu/~olvi/uwmp/cancer.html#diag). Trata-se de um conjunto de dados de classificação binária (benigno/maligno) de diagnóstico de câncer de mama. Este conjunto de dados contém diversas características (*features*) anotadas para cada amostra, bem como seu respectivo rótulo.

Para simplificar o problema e tornar factível a visualização da fronteira de decisão do modelo treinado, trabalharemos com um subconjunto de atributos:
- Raio (média)
- Textura (média)

In [ ]:
# carregando dataset
data = load_breast_cancer(as_frame=True)
# selecionando subconjunto com atributos mencionados acima
X, y = data.data[['mean radius', 'mean texture']], data.target
# salvando nomes dos atributos para utilizar nas visualizações
feature_labels = X.columns.values

# verificando os dados
display(pd.concat([X, y], axis=1))
fig = vis.plot_2d_data(X.to_numpy(), y, feature_labels)
plt.show()

Depois de carregar os dados, devemos escolher um protocolo de avaliação para que possamos mensurar a capacidade de generalização dos modelos treinados. Neste caso, vamos utilizar o protocolo Holdout, separando os dados em conjuntos de treinamento, validação e teste. Alguns pontos importantes:
- Os subconjuntos são gerados de maneira estratificada
- A divisão das amostras entre cada subconjunto é feita de maneira estocástica
- Os conjuntos de validação e teste devem possuir um número de amostras adequado para que seja possível constatar capacidade de generalização

In [ ]:
X_train, X_valid, X_test, y_train, y_valid, y_test = utils.get_dataset_splits(X, y, ratios=[0.6, 0.2, 0.2], seed=123)
print(f'Train:\t{len(X_train)}\t({len(X_train) / len(X):.4f})')
print(f'Valid:\t{len(X_valid)}\t({len(X_valid) / len(X):.4f})')
print(f'Test:\t{len(X_test)}\t({len(X_test) / len(X):.4f})')

Depois de separar os dados, podemos visualizar as distribuições dos valores dos atributos preditivos e atributo alvo, a fim de obtermos uma compreensão maior sobre os dados. Isto pode nos indicar qual procedimento de pré-processamento executar e qual medida de avaliação seria indicada.

In [ ]:
fig = vis.visualize_statistics(X_train, y_train, data.target_names)
plt.show()

Depois de analisar os dados, vamos escolher uma medida de avaliação e montar um *pipeline* de pré-procesamento.

**Avaliação:** os modelos de classificação no Scikit-Learn já possuem uma medida de avaliação padrão em sua interface (classificação: acurácia). Para utilizar métricas diferentes, podemos consultar a [documentação](https://scikit-learn.org/stable/modules/model_evaluation.html).

**Pré-processamento:** É importante ressaltar que o *pipeline* de processamento depende do algoritmo de aprendizado utilizado. Alguns algoritmos não funcionam para determinados tipos de atributos. Além disto, alguns algoritmos são sensíveis à escalas diferentes de atributos, muitas vezes sofrendo perdas significativas de desempenho.

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

## Criando um Classificador

Existem diversos algoritmos de Aprendizado de Máquina para classificação. A escolha de qual algoritmo utilizar depende de uma série de fatores:
- Disponibilidade de recursos computacionais (treino e inferência)
- Complexidade de modelo desejada (interpretabilidade VS desempenho)
- Complexidade do conjunto de dados (número de atributos)
- ...

Neste Notebook, exploraremos diversos algoritmos, aplicando-os na mesma base de dados. Basta (des)comentar as linhas de código correspondentes! Seguem as documentações de cada algoritmo:
- [k-NN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
- [Árvore de Decisão](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)
- [Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)
- [Regressão Logística](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) ([Polinomial](https://scikit-learn.org/stable/modules/linear_model.html#polynomial-regression-extending-linear-models-with-basis-functions))
- [SVMs](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)

Depois de escolhermos um algoritmo, devemos testar diversos valores de hiperparâmetros para melhorar o desempenho do modelo (sempre mantendo em mente a capacidade de generalização!).

In [ ]:
classifier = KNeighborsClassifier(
    n_neighbors=5,
    weights='uniform',
    p=2,
)

# classifier = DecisionTreeClassifier(
#     criterion='gini',
#     max_depth=5,
# )

# classifier = GaussianNB()

# classifier = LogisticRegression(
#     penalty='l2',
#     C=1,
#     random_state=123,
#     max_iter=1000,
#     #solver='liblinear',
# )

# classifier = SVC(
#     C=1,
#     kernel='rbf',
# )


classifier.fit(X_train, y_train)

score_train = classifier.score(X_train, y_train)
score_valid = classifier.score(X_valid, y_valid)
print(f'Accuracy (Train): {score_train:.4f}')
print(f'Accuracy (Valid): {score_valid:.4f}')

## Resultado Final + Fronteira de Decisão

Depois de realizarmos os experimentos para escolha do algoritmo e dos valores de hiperparâmetro, podemos realizar o teste final do modelo no conjunto de dados de teste. Podemos, também, visualizar a fronteira de decisão gerada pelo modelo treinado, a fim de compreender as diferenças entre algoritmos bem como o respectivo impacto de cada hiperparâmetro.

In [ ]:
score_test = classifier.score(X_test, y_test)
print(f'Accuracy (Test): {score_test:.4f}')

fig = vis.plot_2d_boundary(classifier, X_test, y_test, feature_labels, smooth=False)
plt.show()

# Fim do Notebook